In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
shusrith_heateq_path = kagglehub.dataset_download('shusrith/heateq')

print('Data source import complete.')

100%|██████████| 352M/352M [00:05<00:00, 65.2MB/s]

Extracting files...


Data source import complete.


In [2]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm


class PINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList(
            [
                (
                    nn.Linear(input_size if i == 0 else hidden_size, hidden_size)
                    if i % 2 == 0
                    else nn.Tanh()
                )
                for i in range(20)
            ]
        )
        self.layers.append(nn.Linear(hidden_size, output_size))
        self.loss = nn.MSELoss()
        self.alpha = nn.Parameter(torch.tensor([0.1], requires_grad=True).to("cuda"))
        self.optimizer = torch.optim.Adam(self.parameters(), lr=5e-2)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def loss_fn(self, x, u):
        u_pred = self.forward(x)
        return self.loss(u_pred, u)

    def residual_loss(self, xtrain, fhat):
        x = xtrain[:, 0]
        t = xtrain[:, 1]
        g = xtrain.clone()
        g.requires_grad = True
        u_pred = self.forward(g)
        u_x_t = torch.autograd.grad(
            u_pred, g, torch.ones_like(u_pred), create_graph=True
        )[0]
        u_x, u_t = u_x_t[:, 0], u_x_t[:, 1]
        u_xx = torch.autograd.grad(u_x, g, torch.ones_like(u_x), create_graph=True)[0][
            :, 0
        ]
        residual = u_t - self.alpha * u_xx
        return self.loss(residual, fhat)

    def total_loss(self, xtrain, utrain, fhat):
        return self.loss_fn(xtrain, utrain) + self.residual_loss(xtrain, fhat)

    def train_model(self, xtrain, utrain, epochs=1000):
        fhat = torch.zeros(xtrain.shape[0], 1, device="cuda")
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            loss = self.total_loss(xtrain, utrain, fhat)
            loss.backward()
            self.optimizer.step()
            if epoch % 1000 == 0:
                print(f"Epoch {epoch}, Loss {loss.item()}, alpha {self.alpha.item()}")

In [17]:
!ls

data.h5


In [ ]:
model = PINN(input_size=2, hidden_size=20, output_size=1).to("cuda")
print(model)

PINN(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Tanh()
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): Tanh()
    (6): Linear(in_features=20, out_features=20, bias=True)
    (7): Tanh()
    (8): Linear(in_features=20, out_features=20, bias=True)
    (9): Tanh()
    (10): Linear(in_features=20, out_features=20, bias=True)
    (11): Tanh()
    (12): Linear(in_features=20, out_features=20, bias=True)
    (13): Tanh()
    (14): Linear(in_features=20, out_features=20, bias=True)
    (15): Tanh()
    (16): Linear(in_features=20, out_features=20, bias=True)
    (17): Tanh()
    (18): Linear(in_features=20, out_features=20, bias=True)
    (19): Tanh()
    (20): Linear(in_features=20, out_features=1, bias=True)
  )
  (loss): MSELoss()
)


In [3]:
import numpy as np
import h5py
L = 1.0
Nx = 50
dx = L / Nx
T = 0.2
Nt = 500
dt = T / Nt
x = np.linspace(0, L, Nx)
t = np.linspace(0, T, Nt)
x.shape, t.shape

((50,), (500,))

In [4]:
X, T = np.meshgrid(x, t)
xtrue = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
device = "cuda"

In [5]:
import torch
import numpy as np
def loadAndPrep(u):
    idx = np.random.choice(u.flatten().shape[0], 10000, replace=False)
    global xtrue
    xtrain = xtrue[idx, :]
    utrain = u.flatten()[idx][:, None]
    xtrain = torch.tensor(xtrain, dtype=torch.float32, device=device)
    xtrue = torch.tensor(xtrue, dtype=torch.float32, device=device)
    utrain = torch.tensor(utrain, dtype=torch.float32, device=device)
    return xtrain, utrain

In [6]:
import gc
def trainAndLog(u):
    xtrain, utrain = loadAndPrep(u)
    model = PINN(input_size=2, hidden_size=20, output_size=1).to("cuda")
    loss = model.train_model(xtrain, utrain, epochs=15000)
    alpha = model.alpha.item()
    del model
    gc.collect()
    torch.cuda.empty_cache()
    return alpha, loss

In [7]:
!ls /kaggle/input

In [25]:
import random
import json

random.seed(69)
d = {}
with h5py.File("/root/.cache/kagglehub/datasets/shusrith/heateq/versions/1/data.h5", "r") as f:
    a = random.choices(list(f.keys()), k=50)
    n = 0
    for i in a:
        print(n)
        d[i] = []
        e = f[i]["alpha"][()]
        print(e)
        d[i].append({"alpha" : e})
        uclean = f[i]["u"][:].T
        pred, loss = trainAndLog(uclean)
        d[i].append({"clean" : [{"predicted" : pred, "loss" : loss}]})
        unoisy = f[i]["u_noisy"][:]
        pred, loss = trainAndLog(unoisy)
        d[i].append({"noisy": [{"predicted" : pred, "loss" : loss}]})
        n += 1
        with open("results.json", "w") as g:
            json.dump(d, g)

0
0.4469201134477532
Epoch 0, Loss 0.14404895901679993, alpha 0.09904533624649048


<ipython-input-5-247b4b8569b6>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xtrain = torch.tensor(xtrain, dtype=torch.float32, device=device)
<ipython-input-5-247b4b8569b6>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xtrue = torch.tensor(xtrue, dtype=torch.float32, device=device)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([10000, 1])) that is different to the input size (torch.Size([10000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1000, Loss 0.0029497831128537655, alpha 0.11380869150161743
Epoch 2000, Loss 0.0022630449384450912, alpha 0.19058184325695038
Epoch 3000, Loss 0.0019271105993539095, alpha 0.24109554290771484
Epoch 4000, Loss 0.0017500118119642138, alpha 0.2715674042701721
Epoch 5000, Loss 0.001588989864103496, alpha 0.29526445269584656
Epoch 6000, Loss 0.0024964483454823494, alpha 0.31385985016822815
Epoch 7000, Loss 0.0013756247935816646, alpha 0.32687750458717346
Epoch 8000, Loss 0.0012977550504729152, alpha 0.33711546659469604
Epoch 9000, Loss 0.0012384055880829692, alpha 0.3440559208393097
Epoch 10000, Loss 0.0011868656147271395, alpha 0.3487817645072937
Epoch 11000, Loss 0.0011456965003162622, alpha 0.3519802689552307
Epoch 12000, Loss 0.0011563899461179972, alpha 0.35475292801856995
Epoch 13000, Loss 0.0011232774704694748, alpha 0.3568335771560669
Epoch 14000, Loss 0.0010504674864932895, alpha 0.3584868609905243
Epoch 0, Loss 0.23612645268440247, alpha 0.1009959951043129
Epoch 1000, Loss 0

KeyboardInterrupt: 